In [2]:
import pandas as pd
from data_extraction import DataExtractor

extract = DataExtractor()
test = extract.read_rds_table()
print(test)
#test = test.set_index('index')
#print(test)

#index needs to be reconfigured

#test.to_excel('output_file.xlsx', index=True)






       index first_name last_name date_of_birth                       company  \
0          0   Sigfried     Noack    1990-09-30            Heydrich Junitz KG   
1          1        Guy     Allen    1940-12-01                       Fox Ltd   
2          2      Harry  Lawrence    1995-08-02     Johnson, Jones and Harris   
3          3     Darren   Hussain    1972-09-23                   Wheeler LLC   
4          4      Garry     Stone    1952-12-20                    Warner Inc   
...      ...        ...       ...           ...                           ...   
15315  14913    Stephen   Jenkins    1943-08-09  Thornton, Carroll and Newman   
15316  14994    Stephen     Smith    1948-08-20               Robinson-Harris   
15317  15012    Stephen  Losekann    1940-10-09                       Rosenow   
15318  15269    Stephen    Rivera    1952-06-04         Taylor, Fry and Jones   
15319   1249    Stephen    Duncan    1994-03-27    Phillips, Brown and Powell   

                      email

In [26]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15320 entries, 0 to 15319
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   index          15320 non-null  int64 
 1   first_name     15320 non-null  object
 2   last_name      15320 non-null  object
 3   date_of_birth  15320 non-null  object
 4   company        15320 non-null  object
 5   email_address  15320 non-null  object
 6   address        15320 non-null  object
 7   country        15320 non-null  object
 8   country_code   15320 non-null  object
 9   phone_number   15320 non-null  object
 10  join_date      15320 non-null  object
 11  user_uuid      15320 non-null  object
dtypes: int64(1), object(11)
memory usage: 1.4+ MB


- showing no empty cells.
- however "NULL" strings entered in data.
- index column needs dropping.
- use pandas own index as rightful index

- remove duplicates shows 20 duplicates but all NULL NULL NULL meaning total 21 NULL ROWS exist

- rows with false data also exist, need to create a method for removing those instead of individually removing indexes.

- need to remove unwanted characters in addresses, phone numbers etc

- assign datatypes 


In [14]:
test_drop = test.drop('index',axis = 1 )
print(test_drop)

df_cleaned = test_drop.drop_duplicates()
print(df_cleaned.sum())


      first_name last_name date_of_birth                       company  \
0       Sigfried     Noack    1990-09-30            Heydrich Junitz KG   
1            Guy     Allen    1940-12-01                       Fox Ltd   
2          Harry  Lawrence    1995-08-02     Johnson, Jones and Harris   
3         Darren   Hussain    1972-09-23                   Wheeler LLC   
4          Garry     Stone    1952-12-20                    Warner Inc   
...          ...       ...           ...                           ...   
15315    Stephen   Jenkins    1943-08-09  Thornton, Carroll and Newman   
15316    Stephen     Smith    1948-08-20               Robinson-Harris   
15317    Stephen  Losekann    1940-10-09                       Rosenow   
15318    Stephen    Rivera    1952-06-04         Taylor, Fry and Jones   
15319    Stephen    Duncan    1994-03-27    Phillips, Brown and Powell   

                      email_address  \
0                 rudi79@winkler.de   
1      rhodesclifford@henderson.c

first_name       SigfriedGuyHarryDarrenGarryDavidAnneLouisKathl...
last_name        NoackAllenLawrenceHussainStoneTorresMorrisRobe...
date_of_birth    1990-09-301940-12-011995-08-021972-09-231952-1...
company          Heydrich Junitz KGFox LtdJohnson, Jones and Ha...
email_address    rudi79@winkler.derhodesclifford@henderson.comg...
address          Zimmerstr. 1/0\n59015 GießenStudio 22a\nLynne ...
country          GermanyUnited KingdomUnited KingdomUnited King...
country_code     DEGBGBGBGBUSGBGBGBUSGBGBGBDEDEGBUSGBDEGBGBDEGB...
phone_number     +49(0) 047905356(0161) 496 0674+44(0)121 49603...
join_date        2018-10-102001-12-202016-12-162004-02-232006-0...
user_uuid        93caf182-e4e9-4c6e-bebb-60a1a9dcf9b88fe96c3a-d...
dtype: object


In [18]:


# Assuming df is your DataFrame
# df = ...

duplicates = test_drop[test_drop.duplicated(subset=test_drop.columns.difference(['index']))]

total_duplicates = test_drop.duplicated(subset=test_drop.columns.difference(['index'])).sum()

print(f"Duplicate Rows (excluding index): {duplicates}")
print(total_duplicates)

# Display the DataFrame without duplicates
#print("DataFrame after removing duplicates:")
#print(df_no_duplicates)



Duplicate Rows (excluding index):       first_name last_name date_of_birth company email_address address  \
1022        NULL      NULL          NULL    NULL          NULL    NULL   
1805        NULL      NULL          NULL    NULL          NULL    NULL   
2103        NULL      NULL          NULL    NULL          NULL    NULL   
2437        NULL      NULL          NULL    NULL          NULL    NULL   
2739        NULL      NULL          NULL    NULL          NULL    NULL   
2764        NULL      NULL          NULL    NULL          NULL    NULL   
4984        NULL      NULL          NULL    NULL          NULL    NULL   
5307        NULL      NULL          NULL    NULL          NULL    NULL   
6920        NULL      NULL          NULL    NULL          NULL    NULL   
7737        NULL      NULL          NULL    NULL          NULL    NULL   
10013       NULL      NULL          NULL    NULL          NULL    NULL   
10224       NULL      NULL          NULL    NULL          NULL    NULL   
1098

IndexError: invalid index to scalar variable.

In [24]:
# Find rows with "NULL" values in any column
rows_with_null = test[test.apply(lambda row: row.astype(str).str.contains('NULL')).any(axis=1)].count()

print("Rows with 'NULL' values:")
print(rows_with_null)


Rows with 'NULL' values:
index            21
first_name       21
last_name        21
date_of_birth    21
company          21
email_address    21
address          21
country          21
country_code     21
phone_number     21
join_date        21
user_uuid        21
dtype: int64


In [25]:
# Count the total number of "NULL" values in the entire DataFrame
total_null_count = (test == 'NULL').sum().sum()

print("Total number of 'NULL' values in the DataFrame:", total_null_count)


Total number of 'NULL' values in the DataFrame: 231


In [17]:
import pandas as pd
import re

# Assuming your DataFrame is named 'test'

# Define the modified regular expression pattern
pattern = r'\b(?=.*[A-Z])(?=.*\d)[A-Z\d]{10}\b'

# Function to check if any value in a Series matches the pattern
def contains_pattern(series):
    return series.apply(lambda x: bool(re.search(pattern, str(x))))

# Use apply to check if any value in each column matches the pattern
matching_entries = test.apply(contains_pattern)

# Get the rows where at least one value in the row matches the pattern
matching_rows = matching_entries.any(axis=1)

# Print the rows where at least one value matches the pattern
print(f"Matching Rows:")
print(test[matching_rows])

num_rows = matching_rows.sum()
print("--------")
print(num_rows)


Matching Rows:
       index  first_name   last_name date_of_birth     company email_address  \
752      752  PYCLKLLC7I  W350SCUD6R    KBTI7FI7Y3  R7IZUNSQX0    3Q791B3VIY   
1046    1047  GI4C78KWH0  UTB5PPYFG8    OFH8YGZJWN  CA1XGS8GZW    7HSZB429UK   
2995    2997  DPAJNJL6PR  B8ZGN8ZJ84    PQPEUO937L  0YJ2FRMDB4    O5Q6D7FDAF   
3536    3539  E5139W1FJ3  X88YLZX0JX    7KGJ3C5TSW  648S19YUFF    BYU3R2LQF8   
5306    5309  XKDVQD7BH2  4YSEX8AY1Z    RQTF5XSXP4  GKJZ58DTHT    789HWCYC7M   
6420    6426  EI10BU4INS  SH6Y92V6IK    D2OZZHWOLK  LNFBRXXUT4    TFEA83AAHP   
8386    8398  I7IJDAPMIM  LUNRZWUKRB    QTVEU5TR8H  JXKOHA5B8F    JK2J6N4DLG   
9013    9026  TG6PAO2RAB  PIG6VAZUPM    L3E8OV4UAC  ROL0IMK6UH    EIV7XWUJ8I   
10211  10224  VCQL3ML6ZT  HULT86HONJ    TLSTUEIKI0  QSS0MWBGFZ    1WNILGM5MK   
10360  10373  G1SI2RM0KA  67VJEE6DS2    YTC82GP4XE  M4Z6M30SJ3    U1AZKY6URZ   
11366  11381  31XQ1Z35J9  8LR57D7RZ0    O1LIA1MT1N  66V9WU2HU1    XASHXWHN80   
12177  12197  2GU3G97VI1 